In [81]:
import glob
import os
import json
import subprocess
import pandas as pd

In [80]:
# Helper function to load and parse JSON files
def extract_phase_encoding_data(files, file_type):
    data = []
    for file_path in sorted(files):
        file_name = os.path.basename(file_path)
        try:
            with open(file_path, 'r') as file:
                json_data = json.load(file)
                run_name = file_name.split("_")[2]
                if file_type == "fmap":
                    data.append({
                        "run": run_name,
                        "fmap_PhaseEncodingPolarityGE": json_data['info']['PhaseEncodingPolarityGE'],
                        "fmap_PhaseEncodingDirection": json_data['info']['PhaseEncodingDirection']
                    })
                elif file_type == "func":
                    task_name = file_name.split("_")[1]
                    data.append({
                        "task": task_name,
                        "run": run_name,
                        "bold_PhaseEncodingPolarityGE": json_data['info']['PhaseEncodingPolarityGE'],
                        "bold_PhaseEncodingDirection": json_data['info']['PhaseEncodingDirection']
                    })
        except KeyError as e:
            print(f"Missing key {e} in {file_path}. Skipping.")
        except Exception as e:
            print(f"Error processing {file_path}: {e}. Skipping.")
    return data


In [84]:
# def get_dim4(nifti_file):
#     try:
#         # Use FSL's fslval tool to extract dim4
#         result = subprocess.run(["fslval", nifti_file, "dim4"], stdout=subprocess.PIPE, text=True)
#         dim4 = int(result.stdout.strip())
#         return dim4
#     except Exception as e:
#         print(f"Error extracting dim4 for {nifti_file}: {e}")
#         return 1  # Default to 1 if extraction fails

# Generate acqparams.txt for each run
def create_acqparams_per_run(merged_table, output_dir, sub, dim4):
    for run in merged_table["run"].unique():
        run_data = merged_table[merged_table["run"] == run]
        
        acqparams = [
            DIRECTION_MAPPING.get(
                row.get("fmap_PhaseEncodingDirection", row.get("bold_PhaseEncodingDirection", "N/A")),
                "0 0 0 0"
            )
            for _, row in run_data.iterrows()
        ]
        repeated_acqparams = acqparams * dim4
        # Write the acqparams.txt for the current run
        output_path = os.path.join(output_dir, f"{sub}_{run}_acqparams.txt")
        with open(output_path, "w") as file:
            file.write("\n".join(repeated_acqparams))
        # print(f"acqparams.txt created for {run} at: {output_path}")


In [85]:


# Define constants

for sub in ['sub-001', 'sub-002','sub-003']:
    base_dir = f"/Users/heejungj/Documents/projects_local/visualsnow_source"
    fmap_path = f"{base_dir}/{sub}/fmap/{sub}_dir-pa_run-*_epi.json"
    func_path = f"{base_dir}/{sub}/func/{sub}_task-*_run-*_bold.json"
    output_dir = f"{base_dir}/acqparams_per_run/{sub}"

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Phase encoding direction mapping
    DIRECTION_MAPPING = {
        "i": "1 0 0 0.064684",
        "i-": "-1 0 0 0.064684",
        "j": "0 1 0 0.064684",
        "j-": "0 -1 0 0.064684",
        "k": "0 0 1 0.064684",
        "k-": "0 0 -1 0.064684",
    }


    # Extract data
    fmap_files = glob.glob(fmap_path)
    func_files = glob.glob(func_path)

    fmap_data = extract_phase_encoding_data(fmap_files, file_type="fmap")
    func_data = extract_phase_encoding_data(func_files, file_type="func")

    # Convert to DataFrames
    fmap_table = pd.DataFrame(fmap_data)
    func_table = pd.DataFrame(func_data)

    # Merge DataFrames on "Run"
    merged_table = pd.merge(fmap_table, func_table, on="run", how="outer")
    merged_table['sub'] = sub
    # Assume df is your DataFrame
    desired_order = ['sub', 'run', 'task', 'bold_PhaseEncodingPolarityGE', 'bold_PhaseEncodingDirection', 'fmap_PhaseEncodingPolarityGE', 'fmap_PhaseEncodingDirection']
    merged_table = merged_table[desired_order]
    merged_table.to_csv(f'./phase_encoding/{sub}_phase_encoding.tsv', sep='\t', header=True, index=False)
    # merged_table
    # Pretty print merged_table
    print(f"\n\n=== Phase Encoding Table for {sub} ===")
    print(merged_table.to_string(index=False, justify='center'))

    # Call the function to generate acqparams
    with open(fmap_files[0], 'r') as file:
        check_dim4 = json.load(file)
    dim4 = check_dim4['info']['NumberOfTemporalPositions']
    create_acqparams_per_run(merged_table, output_dir, sub, dim4)




=== Phase Encoding Table for sub-001 ===
  sub    run         task      bold_PhaseEncodingPolarityGE bold_PhaseEncodingDirection fmap_PhaseEncodingPolarityGE fmap_PhaseEncodingDirection
sub-001 run-01 task-restclosed          Unflipped                        j-                       Flipped                         j             
sub-001 run-02 task-restclosed          Unflipped                        j-                       Flipped                         j             
sub-001 run-03   task-restopen          Unflipped                        j-                       Flipped                         j             
sub-001 run-04   task-restopen          Unflipped                        j-                       Flipped                         j             


=== Phase Encoding Table for sub-002 ===
  sub    run         task      bold_PhaseEncodingPolarityGE bold_PhaseEncodingDirection fmap_PhaseEncodingPolarityGE fmap_PhaseEncodingDirection
sub-002 run-01 task-restclosed          Unfl